## Univariate one step problem with mlp

In [5]:
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import timeseries_dataset_from_array
import warnings

warnings.filterwarnings("ignore")

# Define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Define parameters
n_input = 2
batch_size = 5

# Prepare dataset using timeseries_dataset_from_array
x_data = series[:-1]  # Exclude the last element for inputs
y_data = series[n_input:]  # Targets start after the first `n_input` elements

dataset = timeseries_dataset_from_array(
    data=x_data,
    targets=y_data,
    sequence_length=n_input,
    sequence_stride=1,
    batch_size=batch_size
)

# Print each data sample
for x_batch, y_batch in dataset:
    print(f"{x_batch.numpy()} => {y_batch.numpy()}")

# Define model
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(dataset, epochs=200, verbose=0)

# Make a one-step prediction out of sample
x_input = array([9, 10]).reshape((1, n_input))
print(x_input)
y_output = model.predict(x_input, verbose=0)
print(y_output)


[[1 2]
 [2 3]
 [3 4]
 [4 5]
 [5 6]] => [3 4 5 6 7]
[[6 7]
 [7 8]
 [8 9]] => [ 8  9 10]
[[ 9 10]]
[[11.447251]]


## Univariate one step problem with LSTM

In [6]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import timeseries_dataset_from_array
import warnings

warnings.filterwarnings("ignore")

# Define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(series)

# Reshape to [10, 1]
n_features = 1
series = series.reshape((len(series), n_features))
print(series)

# Define generator using timeseries_dataset_from_array
n_input = 2
batch_size = 5

dataset = timeseries_dataset_from_array(
    data=series[:-1],  # Exclude the last value for input sequences
    targets=series[n_input:],  # Targets start after the first n_input values
    sequence_length=n_input,
    sequence_stride=1,
    batch_size=batch_size
)

# Print each sample
for x, y in dataset:
    print(f"{x.numpy()} => {y.numpy()}")

# Define model
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(dataset, epochs=400, verbose=0)

# Make a one-step prediction out of sample
x_input = array([9, 10]).reshape((1, n_input, n_features))
y_output = model.predict(x_input, verbose=0)
print(y_output)


[ 1  2  3  4  5  6  7  8  9 10]
[[ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]]
[[[1]
  [2]]

 [[2]
  [3]]

 [[3]
  [4]]

 [[4]
  [5]]

 [[5]
  [6]]] => [[3]
 [4]
 [5]
 [6]
 [7]]
[[[6]
  [7]]

 [[7]
  [8]]

 [[8]
  [9]]] => [[ 8]
 [ 9]
 [10]]
[[10.740629]]


## Multivariate one step problem with LSTM - same data for input and output



In [7]:
from numpy import array, hstack
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import timeseries_dataset_from_array
import warnings

warnings.filterwarnings("ignore")

# Define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
# Reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
print(dataset)

# Define parameters
n_features = dataset.shape[1]
print("n_features = %d" % n_features)
n_input = 2
batch_size = 5

# Create dataset using timeseries_dataset_from_array
data = dataset[:-1]  # Exclude the last row for input sequences
targets = dataset[n_input:]  # Targets start after the first `n_input` rows

dataset_generator = timeseries_dataset_from_array(
    data=data,
    targets=targets,
    sequence_length=n_input,
    sequence_stride=1,
    batch_size=batch_size
)

# Print each sample
for x, y in dataset_generator:
    print(f"{x.numpy()} => {y.numpy()}")

# Define model
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(dataset_generator, epochs=400, verbose=0)

# Make a one-step prediction out of sample
x_input = array([[90, 95], [100, 105]]).reshape((1, n_input, n_features))
y_output = model.predict(x_input, verbose=0)
print(y_output)


[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]
n_features = 2
[[[10 15]
  [20 25]]

 [[20 25]
  [30 35]]

 [[30 35]
  [40 45]]

 [[40 45]
  [50 55]]

 [[50 55]
  [60 65]]] => [[30 35]
 [40 45]
 [50 55]
 [60 65]
 [70 75]]
[[[60 65]
  [70 75]]

 [[70 75]
  [80 85]]

 [[80 85]
  [90 95]]] => [[ 80  85]
 [ 90  95]
 [100 105]]
[[110.573204 115.41678 ]]



## Multivariate one step problem - different data for input and output


In [8]:
from numpy import array, hstack
from keras.utils import timeseries_dataset_from_array
import warnings

warnings.filterwarnings("ignore")

# Define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
out_seq = array([25, 45, 65, 85, 105, 125, 145, 165, 185, 205])

# Reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns for input data
dataset = hstack((in_seq1, in_seq2))
print(dataset)

# Define parameters
n_input = 1
batch_size = 1

# Create dataset using timeseries_dataset_from_array
input_dataset = timeseries_dataset_from_array(
    data=dataset,
    targets=out_seq,
    sequence_length=n_input,
    sequence_stride=1,
    batch_size=batch_size
)

# Print each sample
for x, y in input_dataset:
    print(f"{x.numpy()} => {y.numpy()}")


[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]
[[[10 15]]] => [[25]]
[[[20 25]]] => [[45]]
[[[30 35]]] => [[65]]
[[[40 45]]] => [[85]]
[[[50 55]]] => [[105]]
[[[60 65]]] => [[125]]
[[[70 75]]] => [[145]]
[[[80 85]]] => [[165]]
[[[90 95]]] => [[185]]
[[[100 105]]] => [[205]]


In [9]:
from numpy import array, hstack, insert
from keras.utils import timeseries_dataset_from_array
import warnings

warnings.filterwarnings("ignore")

# Define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
out_seq = array([25, 45, 65, 85, 105, 125, 145, 165, 185, 205])

# Reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns for input data
dataset = hstack((in_seq1, in_seq2))
print(dataset)

# Shift the target sample by one step
out_seq = insert(out_seq, 0, 0)  # Insert a 0 at the start
print(out_seq)
out_seq = out_seq[:-1]  # Remove the last value to align with the inputs
print(out_seq)

# Define parameters
n_input = 1
batch_size = 1

# Create dataset using timeseries_dataset_from_array
input_dataset = timeseries_dataset_from_array(
    data=dataset,
    targets=out_seq,
    sequence_length=n_input,
    sequence_stride=1,
    batch_size=batch_size
)

# Print each sample
for x, y in input_dataset:
    print(f"{x.numpy()} => {y.numpy()}")


[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]
[  0  25  45  65  85 105 125 145 165 185 205]
[  0  25  45  65  85 105 125 145 165 185]
[[[10 15]]] => [0]
[[[20 25]]] => [25]
[[[30 35]]] => [45]
[[[40 45]]] => [65]
[[[50 55]]] => [85]
[[[60 65]]] => [105]
[[[70 75]]] => [125]
[[[80 85]]] => [145]
[[[90 95]]] => [165]
[[[100 105]]] => [185]


 ## Univariate multi-step problem

In [10]:
from numpy import array
from keras.utils import timeseries_dataset_from_array
import warnings

warnings.filterwarnings("ignore")

# Define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
target = array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11]])

# Define parameters
n_input = 2
batch_size = 1

# Create dataset using timeseries_dataset_from_array
input_dataset = timeseries_dataset_from_array(
    data=series[:-1],  # Exclude the last element for input sequences
    targets=target[n_input:],  # Targets start after the first `n_input` elements
    sequence_length=n_input,
    sequence_stride=1,
    batch_size=batch_size
)

# Print each sample
for x, y in input_dataset:
    print(f"{x.numpy()} => {y.numpy()}")


[[1 2]] => [[3 4]]
[[2 3]] => [[4 5]]
[[3 4]] => [[5 6]]
[[4 5]] => [[6 7]]
[[5 6]] => [[7 8]]
[[6 7]] => [[8 9]]
[[7 8]] => [[ 9 10]]
[[8 9]] => [[10 11]]
